In [ ]:
from MFEA_lib.model import AbstractModel, SMP_MFEA, MFEA_base, SMP_DE
from MFEA_lib.model.utils import *
from MFEA_lib.model.utils import TuningModel
import pandas as pd
import numpy as np
import scipy.io
from MFEA_lib.tasks.Benchmark.Funcs import * 
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *


### Tuning Model

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
name_benchmarks = []
for id in range(1, 10):
    # print(id) 
    b, i = WCCI22_benchmark.get_complex_benchmark(4)
    name_benchmarks.append("wcci_2tasks_" + str(id))
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)
name_benchmarks.append("CEC_10_")

<h3> Khởi tạo Tuning Model </h3>
model = TuningModel(model_name = ..., list_parameter  = ..., nb_run = 1) 
<p></p>
-  nb_run: số  lần chạy 
<p></p>
- list_parameter là một list trong đó mỗi phần tử là một tuple 
<p></p>
Ex.một phần tử là lr : ('lr', [0.4, 0.5]) 
<p></p>
ở vị trí đầu tiên của phần tử là tên của tham số 
<p></p>
<p></p>
ở vị trí thú hai của phần tử là danh sách giá trị của tham số đó để tuning
<p></p>
Đối với tham số  như gamma trong sbx : ('crossover', {'gammma': [0.4, 0.6]})
<p></p>
vị trí đầu tiền của phần tử là tên tham số  nhận giá trị của sbx 
<p></p>
ví trị thứ hai của phần tử là một dict với key là tên của các tham số  trong sbx - ở đây là gamma và value là danh sách giá trị gamma cần tuning



<h3> Compile Tuning Model </h3> 
model.compile(
    <p></p>
    ls_benchmark = ... , 
    <p></p>
    benchmark_weights = ..., 
    <p></p>
    name_benchmark = ...., 
    <p></p>
    ls_Indclass = ...., 
    <p></p>
    danh sách tham số  compile cho model cần tuning như smp_mfea, mfea1
    <p></p>
)

<p></p>
- ls_benchmark : dạng list, là danh sách các benchmark cần tuning 
<p></p>
- name_benchmark: dạng list, là danh sách tên benchmark cần tuning  
<p></p>
- ls_IndClass : dạng list, là danh sách IndClass 
<p></p>




<h3> Run Tuning Model </h3> 
model.run(
    <p></p>
    path = ..., 
    <p></p>
    replace_folder = ..., 
    <p></p>
    min_value = ..., 
    <p></p>
    danh sách tham số dành cho fit model nhỏ ví dụ như smp_mfea.fit

)
<p></p>
- path: str, là đường dẫn để lưu model và kết quả tuning
<p></p>
- replace_folder = True: dạng boolean: True nếu cho phép ghi đè lên thư mục nếu thư mục tồn tại ở đường dẫn 
<p></p>
- min_value = 0 or 1e-6: những giá trị nhỏ hơn min_value sẽ được xem bằng 0 khi đem đi so sánh kết quả giữa các mô hình 
<p></p>



In [ ]:
model = TuningModel(
    model_name= SMP_MFEA, 
    
    list_parameter= [
        ('crossover', {
            'gamma': [0.4, 0.6]
        }), 
        ('lr', [0.04, 0.05]),
        ('crossover', {
            'alpha': [5, 7]
        }), 
    ],
)
model.compile(
    ls_benchmark = ls_benchmark, 
    benchmark_weights = (np.ones(shape= (3,)) / 3).tolist(), 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,

    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    mutation = GaussMutation(scale = 0.01),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation']
)

fit, com, result = model.run(
    replace_folder=True,
    min_value = 1e-6,
    
    nb_generations= 5, 
    nb_inds_each_task= 100, 
    nb_inds_min= 100,
    lr = 0.05,  
    p_const_intra= 0, 
    p_mutate= 0.1, 
    prob_search = 0.,
    nb_epochs_stop= 1000, 
    swap_po= False,
    evaluate_initial_skillFactor= True
)

### MultBenchmark Model 

<h3> Khởi tạo MultiBenchmark Model </h3> 
<p></p>
smpModel = MultiBenchmark(
<p></p>
    ls_benchmark = ls_benchmark, 
<p></p>
    name_benchmark = [str(i) for i in range(1,4)],
<p></p>
    ls_IndClass = ls_IndClass,
<p></p>
    model= SMP_MFEA
<p></p>
)
<p></p>
- ls_benchmark : dạng list, là danh sách các benchmark cần chạy nhiều lần
<p></p>
- name_benchmark : dạng list, là danh sách tên benchmark 
<p> </p> 
- ls_IndClass : dạng list, là danh sách IndClass tương ứng với benchmark 
<p> </p>
 



<h3> Compile, fit, run Model </h3> 
- compile : các tham số cho model như mfea, smp chạy hàm compile  
<p></p>
- fit: các tham số cho model như smp chạy hàm fit
<p> </p> 
- run: gồm nb_run: số lần chạy một mô hình , 
save_path: đường dẫn thuư mục để lưu model 


In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 3): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

smpModel = MultiBenchmark(
    ls_benchmark = ls_benchmark, 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,
    model= SMP_MFEA
    )

smpModel.compile(
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 10, nb_inds_each_task= 100, nb_inds_min= 100,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

<h3> print result </h3> 

print_attr : dạng list, là tên các giá trị trung bình muốn in ra 
<p> </p> 
print_name : True nếu muốn in tên bộ benchmark 
<p> </p> 
print_time : True nếu muốn in thời gian chạy từng bộ benchmark 
<p> </p> 
print_name_attr: True nếu muốn in tên của từng giá trị trung bình

In [ ]:
smpModel.print_result(print_attr = ['history_cost'],print_name = True, print_time = False, print_name_attr = False)